# En este notebook se muestra una manera de descargar y juntar los archivos descargados en uno solo (el cual servirá tanto para generar los embeddings como para que sea la base para el etiquetado)

In [1]:
from Discriminamometro import Utileria, Discriminamometro
import pandas as pd
import os


obj_Util = Utileria()
obj_Discr = Discriminamometro()

s3 = obj_Util.CrearConexionS3()

bucket='discriminamometro'

In [2]:
def get_s3_keys(bucket):
    """Get a list of keys in an S3 bucket."""
    keys = []
    resp = s3.list_objects_v2(Bucket=bucket)
    for obj in resp['Contents']:
        keys.append(obj['Key'])
    return keys

In [3]:
def get_matching_s3_keys(bucket, prefix='', suffix=''):
    """
    Generate the keys in an S3 bucket.

    :param bucket: Name of the S3 bucket.
    :param prefix: Only fetch keys that start with this prefix (optional).
    :param suffix: Only fetch keys that end with this suffix (optional).
    """
    #s3 = boto3.client('s3')
    kwargs = {'Bucket': bucket}

    # If the prefix is a single string (not a tuple of strings), we can
    # do the filtering directly in the S3 API.
    if isinstance(prefix, str):
        kwargs['Prefix'] = prefix

    while True:

        # The S3 API response is a large blob of metadata.
        # 'Contents' contains information about the listed objects.
        resp = s3.list_objects_v2(**kwargs)
        for obj in resp['Contents']:
            key = obj['Key']
            if key.startswith(prefix) and key.endswith(suffix):
                yield key

        # The S3 API is paginated, returning up to 1000 keys at a time.
        # Pass the continuation token into the next response, until we
        # reach the final page (when this field is missing).
        try:
            kwargs['ContinuationToken'] = resp['NextContinuationToken']
        except KeyError:
            break

In [4]:
# Creating the first Dataframe using dictionary 
pd_Vacio = pd.DataFrame({"user_id":[], "name":[], "screen_name":[], "full_text":[], "location":[]})

pd_Apariencia = pd.DataFrame({"user_id":[], "name":[], "screen_name":[], "full_text":[], "location":[]})
pd_Discapacidad = pd.DataFrame({"user_id":[], "name":[], "screen_name":[], "full_text":[], "location":[]})
pd_Edad = pd.DataFrame({"user_id":[], "name":[], "screen_name":[], "full_text":[], "location":[]})
pd_Genero = pd.DataFrame({"user_id":[], "name":[], "screen_name":[], "full_text":[], "location":[]})
pd_Ideologia = pd.DataFrame({"user_id":[], "name":[], "screen_name":[], "full_text":[], "location":[]})
pd_Orientacion = pd.DataFrame({"user_id":[], "name":[], "screen_name":[], "full_text":[], "location":[]})
pd_Religion = pd.DataFrame({"user_id":[], "name":[], "screen_name":[], "full_text":[], "location":[]})
pd_Neutro = pd.DataFrame({"user_id":[], "name":[], "screen_name":[], "full_text":[], "location":[]})

pd_Todos = pd.DataFrame({"user_id":[], "name":[], "screen_name":[], "full_text":[], "location":[]})

In [5]:
def juntarArchivos(pd_Concentrador, str_Prefijo):
   
    i = 1
    list_Dias = [10]
    for dia in list_Dias:
        for key in get_matching_s3_keys(bucket=bucket, prefix=(str_Prefijo+str(dia))):
            print(key)

            str_LocalFile = 'tmp'+str(i) 
            with open(str_LocalFile, 'wb') as f:
                s3.download_fileobj(bucket, key, f)

            pd = obj_Discr.Utileria.AbrirJSONComoDataframe(str_LocalFile)
            pd_Concentrador=pd_Concentrador.append(pd)
            os.remove(str_LocalFile)

            i = i + 1
        
    return pd_Concentrador
    

In [6]:

pd_Apariencia = juntarArchivos(pd_Vacio, 'apariencia/json_apariencia_202007')
pd_Discapacidad = juntarArchivos(pd_Vacio, 'discapacidad/json_discapacidad_202007')
pd_Edad = juntarArchivos(pd_Vacio, 'edad/json_edad_202007')
pd_Genero = juntarArchivos(pd_Vacio, 'genero/json_genero_202007')
pd_Ideologia = juntarArchivos(pd_Vacio, 'ideologia/json_ideologia_202007')
pd_Orientacion = juntarArchivos(pd_Vacio, 'orientacion/json_orientacion_202007')
pd_Religion = juntarArchivos(pd_Vacio, 'religion/json_religion_202007')
pd_Neutro = juntarArchivos(pd_Vacio, 'neutro/json_neutro_202007')


apariencia/json_apariencia_20200710_000011207443.txt
apariencia/json_apariencia_20200710_001510875633.txt
apariencia/json_apariencia_20200710_003011381984.txt
apariencia/json_apariencia_20200710_004511245039.txt
apariencia/json_apariencia_20200710_010010757449.txt
apariencia/json_apariencia_20200710_011510716203.txt
apariencia/json_apariencia_20200710_013010775473.txt
apariencia/json_apariencia_20200710_014511812088.txt
apariencia/json_apariencia_20200710_020011313726.txt
apariencia/json_apariencia_20200710_021510969812.txt
apariencia/json_apariencia_20200710_023011348511.txt
apariencia/json_apariencia_20200710_024511355545.txt
apariencia/json_apariencia_20200710_030010749995.txt
apariencia/json_apariencia_20200710_031511434729.txt
apariencia/json_apariencia_20200710_033010454890.txt
apariencia/json_apariencia_20200710_034511178585.txt
apariencia/json_apariencia_20200710_040011135730.txt
apariencia/json_apariencia_20200710_041511682736.txt
apariencia/json_apariencia_20200710_0430117142

discapacidad/json_discapacidad_20200710_140022124237.txt
discapacidad/json_discapacidad_20200710_141521406983.txt
discapacidad/json_discapacidad_20200710_143021785053.txt
discapacidad/json_discapacidad_20200710_144523045619.txt
discapacidad/json_discapacidad_20200710_150023250952.txt
discapacidad/json_discapacidad_20200710_151522278640.txt
discapacidad/json_discapacidad_20200710_153022600659.txt
discapacidad/json_discapacidad_20200710_154522284060.txt
discapacidad/json_discapacidad_20200710_160021603548.txt
discapacidad/json_discapacidad_20200710_161521827535.txt
discapacidad/json_discapacidad_20200710_163022532918.txt
discapacidad/json_discapacidad_20200710_164522073143.txt
discapacidad/json_discapacidad_20200710_170021037570.txt
discapacidad/json_discapacidad_20200710_171522056830.txt
discapacidad/json_discapacidad_20200710_173021689370.txt
discapacidad/json_discapacidad_20200710_174521829517.txt
discapacidad/json_discapacidad_20200710_180022405442.txt
discapacidad/json_discapacidad_

genero/json_genero_20200710_123038689223.txt
genero/json_genero_20200710_124538586052.txt
genero/json_genero_20200710_130038543040.txt
genero/json_genero_20200710_131540805789.txt
genero/json_genero_20200710_133040035391.txt
genero/json_genero_20200710_134539320068.txt
genero/json_genero_20200710_140042030831.txt
genero/json_genero_20200710_141540684475.txt
genero/json_genero_20200710_143041378145.txt
genero/json_genero_20200710_144542480039.txt
genero/json_genero_20200710_150043627613.txt
genero/json_genero_20200710_151542131421.txt
genero/json_genero_20200710_153041413757.txt
genero/json_genero_20200710_154542408824.txt
genero/json_genero_20200710_160040608020.txt
genero/json_genero_20200710_161541284028.txt
genero/json_genero_20200710_163042167672.txt
genero/json_genero_20200710_164542524082.txt
genero/json_genero_20200710_170039294168.txt
genero/json_genero_20200710_171541101655.txt
genero/json_genero_20200710_173040131126.txt
genero/json_genero_20200710_174540140147.txt
genero/jso

orientacion/json_orientacion_20200710_063055829281.txt
orientacion/json_orientacion_20200710_064558725662.txt
orientacion/json_orientacion_20200710_070056907012.txt
orientacion/json_orientacion_20200710_071557520738.txt
orientacion/json_orientacion_20200710_073056092506.txt
orientacion/json_orientacion_20200710_074557484615.txt
orientacion/json_orientacion_20200710_080057958786.txt
orientacion/json_orientacion_20200710_081557164902.txt
orientacion/json_orientacion_20200710_083056123363.txt
orientacion/json_orientacion_20200710_084553902035.txt
orientacion/json_orientacion_20200710_090055477094.txt
orientacion/json_orientacion_20200710_091555740118.txt
orientacion/json_orientacion_20200710_093055154182.txt
orientacion/json_orientacion_20200710_094554005399.txt
orientacion/json_orientacion_20200710_100056824616.txt
orientacion/json_orientacion_20200710_101557563962.txt
orientacion/json_orientacion_20200710_103058554835.txt
orientacion/json_orientacion_20200710_104554229450.txt
orientacio

neutro/json_neutro_20200710_000114663890.txt
neutro/json_neutro_20200710_001611923141.txt
neutro/json_neutro_20200710_003117526394.txt
neutro/json_neutro_20200710_004616407955.txt
neutro/json_neutro_20200710_010114627840.txt
neutro/json_neutro_20200710_011612542743.txt
neutro/json_neutro_20200710_013113135078.txt
neutro/json_neutro_20200710_014613897845.txt
neutro/json_neutro_20200710_020114636212.txt
neutro/json_neutro_20200710_021613283491.txt
neutro/json_neutro_20200710_023113171791.txt
neutro/json_neutro_20200710_024611494053.txt
neutro/json_neutro_20200710_030113388716.txt
neutro/json_neutro_20200710_031612687881.txt
neutro/json_neutro_20200710_033110215200.txt
neutro/json_neutro_20200710_034612832752.txt
neutro/json_neutro_20200710_040113347658.txt
neutro/json_neutro_20200710_041616268270.txt
neutro/json_neutro_20200710_043115139967.txt
neutro/json_neutro_20200710_044617676507.txt
neutro/json_neutro_20200710_050116688642.txt
neutro/json_neutro_20200710_051616930703.txt
neutro/jso

In [7]:
pd_Apariencia

,user_id,name,screen_name,full_text,location
0,2.597167e+09,Rigo,_pinche_gordo,"Las plebes si pueden decir ""quiero a uno barbo...",Apatzingan
1,1.025617e+08,JORGE A. LARA,georgelara,@TenochHuerta @Viri_Rios Aqui un par de imbeci...,"Ontario, CA"
2,1.157329e+09,Biebs.,Titacc14,"A pesar de que el 2020 ha sido jodido , nos ha...",Venezuela
3,1.989619e+08,ParanoidBrian.,paranoidbrian,De acuerdo a los datos sobre si eres clase me...,
4,7.928899e+07,Sandra FF2019,sandra_rosarina,"@caireydecopas ESTA JODIDO , OJALA QUE LLEGUEMOS",Argentina
...,...,...,...,...,...
370,2.662896e+08,McLovin,mikeeglez,129. Los pinches gueritos. Y que la gente les ...,Sam's Town
371,2.828943e+09,Super Vrg Ardiente Para Ti.,LindaDm01,"Also, Auron no solto el jodido cuchillo ni cua...",Marte
372,2.520866e+09,Rababococo Deprimido,MC_Raavi,"Also, Auron no solto el jodido cuchillo ni cua...",F
373,2.472496e+09,Kevs,KevinCorral7,Ya perdi todo mi progreso en el Gym a la vvvve...,"Chihuahua, Mexico"


In [8]:
# Quitamos las columnas que no nos interesan
list_ColumnasDrop = ['user_id', 'name','screen_name','location']

pd_Apariencia=pd_Apariencia.drop(list_ColumnasDrop, axis=1)
pd_Discapacidad=pd_Discapacidad.drop(list_ColumnasDrop, axis=1)
pd_Edad=pd_Edad.drop(list_ColumnasDrop, axis=1)
pd_Genero=pd_Genero.drop(list_ColumnasDrop, axis=1)
pd_Ideologia=pd_Ideologia.drop(list_ColumnasDrop, axis=1)
pd_Orientacion=pd_Orientacion.drop(list_ColumnasDrop, axis=1)
pd_Religion=pd_Religion.drop(list_ColumnasDrop, axis=1)

pd_Neutro=pd_Neutro.drop(list_ColumnasDrop, axis=1)

In [9]:
pd_Apariencia['categoria'] = 'apariencia'
pd_Discapacidad['categoria'] = 'discapacidad'
pd_Edad['categoria'] = 'edad'
pd_Genero['categoria'] = 'genero'
pd_Ideologia['categoria'] = 'ideologia'
pd_Orientacion['categoria'] = 'orientacion'
pd_Religion['categoria'] = 'religion'

pd_Neutro['categoria'] = 'neutro'

In [10]:
# Juntamos todos los dataset en uno solo
pd_Todos = pd.DataFrame(columns=['full_text','categoria'])

pd_Todos = pd_Todos.append(pd_Apariencia)
pd_Todos = pd_Todos.append(pd_Discapacidad)
pd_Todos = pd_Todos.append(pd_Edad)
pd_Todos = pd_Todos.append(pd_Genero)
pd_Todos = pd_Todos.append(pd_Ideologia)
pd_Todos = pd_Todos.append(pd_Orientacion)
pd_Todos = pd_Todos.append(pd_Religion)
pd_Todos = pd_Todos.append(pd_Neutro)



In [11]:
pd_Todos = 	pd_Todos.drop_duplicates()
pd_Todos

,full_text,categoria
0,"Las plebes si pueden decir ""quiero a uno barbo...",apariencia
1,@TenochHuerta @Viri_Rios Aqui un par de imbeci...,apariencia
2,"A pesar de que el 2020 ha sido jodido , nos ha...",apariencia
3,De acuerdo a los datos sobre si eres clase me...,apariencia
4,"@caireydecopas ESTA JODIDO , OJALA QUE LLEGUEMOS",apariencia
...,...,...
360,Cuando sale Blumel? https://t.co/1dl3nFtOQa,neutro
361,Empezamos buscando donde comprar cubrebocas y ...,neutro
363,"Que conste que se les dijo, y se nos tacho de ...",neutro
364,Que afortunado me siento de tener a un amigo q...,neutro


In [18]:
pd_Todos.to_csv('tweets_agosto_prueba.csv', encoding='utf-8', sep=',', index=False, line_terminator='\n')

In [21]:
pd_Todos2 = pd.read_csv('tweets_agosto_prueba.csv',lineterminator='\n')